In [2]:
import torch as th
import numpy as np
import torch.nn.functional as F
import tensorflow as tf
from torch.autograd import Variable
import gym

In [20]:
a = np.array([0,1])
b = F.one_hot(th.tensor(a,dtype=th.int64))
b


tensor([[1, 0],
        [0, 1]])

In [19]:
a = np.array([0,1])
a = th.tensor(a,dtype=th.int64)
a

tensor([0, 1])

In [53]:
x = [1,2,3,4,4]
u = [0.6982, 0.8768, 0.5175, 0.5463, 0.9251]
tf.nn.softmax(x - tf.math.log(-tf.math.log(u)), axis = -1)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.00854149, 0.06344201, 0.03441923, 0.10194238, 0.7916549 ],
      dtype=float32)>

In [46]:
x - tf.math.log(-tf.math.log(u))

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([1.1881278, 2.990991 , 4.569584 , 6.131401 , 4.519613 ],
      dtype=float32)>

In [51]:
u = th.rand(len(x))
x = th.tensor([1,2,3,4,4])
input = x - th.log(-th.log(u))
F.softmax(input)

C:\Users\77189\AppData\Local\Temp\ipykernel_18736\1652972382.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(input)


tensor([0.0085, 0.0635, 0.0344, 0.1020, 0.7916])

In [57]:
tf.shape([1.0])

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>

In [61]:
env = gym.make("merge-v0")

# env.agent_num = 2
    
# env.leader_num = 1
# env.follower_num = 1
# env.num_state

UnregisteredEnv: No registered env with id: merge-v0

In [ ]:
from leader_agent import LeaderAgent
a = LeaderAgent(
        env_specs=env.env_specs,
        policy=Actor(dim_observation=env.num_state,
                     dim_action=env.action_num),
        qf=Critic(
            dim_observation=env.num_state,
            dim_action=env.action_num
        ),
        replay_buffer=IndexedReplayBuffer(observation_dim=env.num_state,
                                          action_dim=env.action_num,
                                          opponent_action_dim=env.agent_num,
                                          next_observation_dim = env.num_state,
                                          max_replay_buffer_size=max_replay_buffer_size
                                          ),
        agent_id=agent_id,
    )

In [1]:
import numpy as np
import torch
from scipy.optimize import minimize

# Define the lower-level objective function as a neural network using PyTorch
def lower_level_obj(z, x, neural_network_model, device):
    z = torch.tensor(z, dtype=torch.float32, requires_grad=True, device=device)
    z = z.unsqueeze(0)
    y = neural_network_model(z)
    obj = y[0, 0]
    obj.backward()
    grad = z.grad.cpu().numpy().flatten()
    return obj.item(), grad

def upper_level_obj(x, lower_level_obj, x_val, neural_network_model, device):
    res = minimize(lower_level_obj, x_val, args=(x, neural_network_model, device), method='L-BFGS-B')
    return res.fun

def upper_level_grad(x, lower_level_obj, x_val, neural_network_model, device):
    res = minimize(lower_level_obj, x_val, args=(x, neural_network_model, device), method='L-BFGS-B')
    return res.jac

def bilevel_optimization(x0, x_val, neural_network_model, bounds, device):
    res = minimize(upper_level_obj, x0, jac=upper_level_grad,
                   args=(lower_level_obj, x_val, neural_network_model, device),
                   method='L-BFGS-B', bounds=bounds)
    return res.x


In [3]:
costs = np.zeros(2)
costs

array([0., 0.])

In [15]:
x = th.tensor([2.0], requires_grad = True)
y = th.tensor([2.0], requires_grad = True)

a = th.cat((x,y))

loss = 3*a
loss = th.sum(2*loss**2) 
loss.backward()
x.grad

tensor([72.])

In [21]:
x = Variable(th.randn(5), requires_grad=True)
x.shape[0]

5

In [ ]:
import torch

def leader_obj(x, y):
    """Leader's objective function"""
    return torch.sum(x**2) + torch.sum(y**2)

def follower_obj(x, y, z):
    """Follower's objective function"""
    return torch.sum(x**2 + y**2) - z

def solve_stackelberg(num_iter, learning_rate):
    x = torch.tensor([1.0, 2.0], requires_grad=True)
    y = torch.tensor([3.0, 4.0], requires_grad=True)
    z = torch.tensor([0.0], requires_grad=True)
    
    for i in range(num_iter):
        # First solve the follower's problem
        opt_follower = torch.optim.SGD([x, y, z], lr=learning_rate)
        opt_follower.zero_grad()
        loss_follower = -follower_obj(x, y, z)
        loss_follower.backward()
        opt_follower.step()
        
        # Then solve the leader's problem
        opt_leader = torch.optim.SGD([x, y], lr=learning_rate)
        opt_leader.zero_grad()
        loss_leader = leader_obj(x, y)
        loss_leader.backward()
        opt_leader.step()
    
    return x, y, z

# Solve the Stackelberg problem
x_sol, y_sol, z_sol = solve_stackelberg(num_iter=100, learning_rate=0.1)
print("x_sol:", x_sol)
print("y_sol:", y_sol)
print("z_sol:", z_sol)


In [4]:
leader_action = Variable(th.randn(5), requires_grad=True)
leader_action

tensor([-0.4398, -0.3162,  0.8232, -0.8618,  0.0596], requires_grad=True)

In [6]:
a = th.randn(5,requires_grad=True)
a

tensor([-0.5247,  0.2624,  0.6297, -0.2009,  1.7447], requires_grad=True)